In [8]:
!pip install findspark

In [9]:
import findspark
findspark.init("/home/ec2-user/spark-2.4.5-bin-hadoop2.7/")
import pyspark
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext, Row
from pyspark.sql.types import StructType, StructField, DoubleType, StringType, IntegerType
from pyspark.sql.functions import udf
from pyspark.ml.feature import CountVectorizerModel, IDFModel, StandardScalerModel, Tokenizer
from pyspark.ml.classification import LogisticRegressionModel
import boto3
import time
 
# Wait for 5 seconds


sqs = boto3.resource('sqs')

APP_NAME = "BigData"
conf = pyspark.SparkConf().setAll([('spark.app.name', APP_NAME)])
sc = SparkContext(conf=conf)
sqlc = SQLContext(sc)


In [12]:
def to_ngram(payload_obj):
    n=2
    payload = str(payload_obj)
    ngrams = ''
    for i in range(0,len(payload)-n + 1):
        ngrams += payload[i:i+n]+ ' '
    return ngrams[:-1]


# define a function to compute sentiments of the received tweets
def get_prediction(queries):
    try:
        queries = queries.map(lambda w: Row(payload=w))
        queries = sqlc.createDataFrame(queries)

        queries = queries.withColumn('ngrams', ngrams(queries['payload']))
        queries = tokenizer.transform(queries)
        queries = vectorizer.transform(queries)
        queries = idf_model.transform(queries)
        queries = scalerModel.transform(queries)
        preds = model.transform(queries)
        preds.select('payload','prediction').show()

    except : 
        print('No data')


    
ngrams = udf(to_ngram, StringType())
tokenizer = Tokenizer.load('../models/Tokenizer')
vectorizer = CountVectorizerModel.load('../models/Vectorizer')
idf_model = IDFModel.load('../models/idf')
scalerModel = StandardScalerModel.load('../models/scalerModel')
model = LogisticRegressionModel.load('../Logistic_Regression_Model')


In [15]:
def recieve():
    queue = sqs.get_queue_by_name(QueueName='test')
    while True:
        messages = queue.receive_messages()
        if len(messages) == 0:
            print('No data')
        else:
            queries = []
            for message in messages:
                # Print out the body and author (if set)
                print('query: {0}'.format(message.body))
                # Let the queue know that the message is processed
                queries += [message.body]
                message.delete()
            
            queries = sc.parallelize(queries)
            queries = queries.map(lambda w: Row(payload=w))
            queries = sqlc.createDataFrame(queries)

            queries = queries.withColumn('ngrams', ngrams(queries['payload']))
            queries = tokenizer.transform(queries)
            queries = vectorizer.transform(queries)
            queries = idf_model.transform(queries)
            queries = scalerModel.transform(queries)
            preds = model.transform(queries)
            preds.select('payload','prediction').show()
        time.sleep(2)

recieve()

query: hello!
+-------+----------+
|payload|prediction|
+-------+----------+
| hello!|       0.0|
+-------+----------+

No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
query: kdkdkdkdk
+---------+----------+
|  payload|prediction|
+---------+----------+
|kdkdkdkdk|       0.0|
+---------+----------+

No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data


KeyboardInterrupt: 